#### As usual, there are two method
- [Core method](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html#tutorial-core-insert)
- [ORM method](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)

#### here illustrated orm method

#### refresher/prerequisite to the orm method
- [Executing with an ORM Session - introduces how to make an ORM Session object](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)
- [Using ORM Declarative Forms to Define Table Metadata - where we set up our ORM mappings of the User and Address entities](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-orm-table-metadata)
- [Selecting ORM Entities and Columns - a few examples on how to run SELECT statements for entities like User](https://docs.sqlalchemy.org/en/20/tutorial/data_select.html#tutorial-selecting-orm-entities)

In [1]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey

from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

class User(Base):
    
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"
    
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"

engine_config = template_engine.format(
        db_username="a",
        db_password="b",
        db_host="c-db.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="")

engine = create_engine(engine_config,
    echo=True
) 


### Adding objects to a session

In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    
    squidward = User(name="squidward", fullname="Squidward Tentacles")
    krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")
    session.add(squidward)
    session.add(krabs)
    #session.commit()
    
    ### check the state of Session using `Session.new`
    print(session.new)
    
    print("Before flushing")
    print(squidward.id)
    print(krabs.id)
    
    
    ###flushing (create sql queries)
    session.flush()
    
    ### autogenerated id attributes
    print("After flushing")
    print(squidward.id)
    print(krabs.id)
    
    ### transaction from above remain open until Session.commit() or Session.rollback() or Session.close() being calle    
    session.commit()

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])
Before flushing
None
None
2023-03-06 10:29:19,445 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-06 10:29:19,446 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 10:29:19,470 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-06 10:29:19,471 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 10:29:19,484 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-06 10:29:19,485 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-06 10:29:19,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:29:19,531 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name__0)s::VARCHAR(30), %(fullname__0)s::VARCHAR), (%(name__1)s::VARCHAR(30), %(fullname__1)s::VARCHAR) RETURNING user_account.id
2023-03-06 10:29:19,532 INFO sqlalchemy.engine.Engine [generated in 0.00024s (ins

### SELECT 

In [9]:
from sqlalchemy import select

with Session(engine) as session:

    squidwardclone = session.get(User, 10)
    
    print(squidwardclone)

    #select with filter
    ehkrabsclone = session.execute(select(User).filter_by(name="ehkrabs"))
    
    for item in ehkrabsclone:
        print(item)

2023-03-06 10:30:27,542 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:30:27,545 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = %(pk_1)s::INTEGER
2023-03-06 10:30:27,547 INFO sqlalchemy.engine.Engine [cached since 67.97s ago] {'pk_1': 10}
None
2023-03-06 10:30:27,570 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s::VARCHAR(30)
2023-03-06 10:30:27,571 INFO sqlalchemy.engine.Engine [cached since 67.98s ago] {'name_1': 'ehkrabs'}
(User(id=15, name='ehkrabs', fullname='Eugene H. Krabs'),)
(User(id=17, name='ehkrabs', fullname='Eugene H. Krabs'),)
2023-03-06 10:30:27,579 INFO sqlalchemy.engine.Engine ROLLBACK


if not used within `with`
it's best to explicitly close the Session when we are done with it.

`session.close()`